In [ ]:
import pandas as pd
df=pd.read_csv('P1.csv')
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns
plt.figure(figsize=(12, 12))
sns.boxplot(data=df)
plt.xticks(rotation=90)  # Rotaciona os rótulos do eixo x para melhor visualização
plt.show()

In [ ]:
def find_outliers_iqr(data):
    outliers_dict = {}
    for column in data.columns:
        if data[column].dtype in ['int64', 'float64']:  
            Q1 = data[column].quantile(0.25)
            Q3 = data[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
            outliers_dict[column] = outliers[column]
    return outliers_dict

outliers = find_outliers_iqr(df)

for column, outliers_data in outliers.items():
    print(f"Outliers na coluna '{column}':")
    print(outliers_data)
    print("\n")

#### Criação Label

In [ ]:
df['erro_x'] =df['Approx_x']-df['True_x'] 
df['erro_y']=df['Approx_y']-df['True_y'] 
df['erro_z']=df['Approx_z']-df['True_z']
df['erro_vx']=df['Approx_vx']-df['True_vx']
df['erro_vy']=df['Approx_vy']-df['True_vy']
df['erro_vz']=df['Approx_vz']-df['True_vz']

In [ ]:
df.head()

In [ ]:
df_time_0 = df[df['Time'] == 0].head(5)
df_time_0 

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

columns_to_select = ['Delta_t', 'c1_z', 'c1_t', 'c2_z', 'c2_t', 'c3_z', 'c3_t', 'c4_z', 'c4_t', 'c5_z', 'c5_t', 'erro_z'
                     ]#'c1_x', 'c2_x', 'c3_x', 'c4_x', 'c5_x', 'c1_y', 'c2_y', 'c3_y', 'c4_y', 'c5_y']
selected_columns_df = df[columns_to_select]

X = selected_columns_df[['Delta_t', 'c1_z', 'c1_t', 'c2_z', 'c2_t', 'c3_z', 'c3_t', 'c4_z', 'c4_t', 'c5_z', 'c5_t'
                         ]]#'c1_x', 'c2_x', 'c3_x', 'c4_x', 'c5_x', 'c1_y', 'c2_y', 'c3_y', 'c4_y', 'c5_y']]
poly = PolynomialFeatures(degree=5, include_bias=False)

y = selected_columns_df['erro_z']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.model_selection import GroupShuffleSplit
groups=df['Time']
gss = GroupShuffleSplit(n_splits=1, test_size=.2, random_state=42)
for i, (train_index, test_index) in enumerate(gss.split(X, y, groups)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}, group={groups[train_index]}")
    print(f"  Test:  index={test_index}, group={groups[test_index]}")

X_train=X.iloc[train_index]
X_test=X.iloc[test_index]
y_train=y.iloc[train_index]
y_test=y.iloc[test_index]


Fold 0:
  Train: index=[    0     1     2 ... 80637 80638 80639], group=0              0.0
1              0.0
2              0.0
3              0.0
4              0.0
           ...    
80635    1684800.0
80636    1684800.0
80637    1684800.0
80638    1684800.0
80639    1684800.0
Name: Time, Length: 64512, dtype: float64
  Test:  index=[ 8064  8065  8066 ... 76605 76606 76607], group=8064      172800.0
8065      172800.0
8066      172800.0
8067      172800.0
8068      172800.0
           ...    
76603    1598400.0
76604    1598400.0
76605    1598400.0
76606    1598400.0
76607    1598400.0
Name: Time, Length: 16128, dtype: float64


In [49]:
from sklearn.preprocessing import StandardScaler


X_train_poly=poly.fit_transform(X_train)
X_test_poly=poly.transform(X_test)

scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train_poly)
X_test_norm = scaler.transform(X_test_poly)
X_test_norm.shape



(16128, 4367)

In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb

def regressao_linear(X_train_norm,y_train):
    model = LinearRegression()
    model.fit(X_train_norm, y_train)
    return model

def RandomForest(X_train_norm, y_train, n_estimators=50):
    model=RandomForestRegressor(n_estimators=n_estimators, random_state=42)
    model.fit(X_train_norm, y_train)
    return model

def ada_boost(X_train_norm, y_train):
    model = AdaBoostRegressor()
    model.fit(X_train_norm,y_train)
    return model

def decision_tree(X_train_norm,y_train, max_depth=5):
    model = DecisionTreeRegressor(max_depth=max_depth, random_state=42)
    model.fit(X_train_norm,y_train)
    return model

def xboost(X_train_norm, y_train):
    model=xgb.XGBRegressor() 
    model.fit(X_train_norm, y_train)
    return model


resultado_reg=regressao_linear(X_train_norm, y_train)
# resultado_randomForest=RandomForest(X_train_norm, y_train)
# resultado_adaBoost=ada_boost(X_train_norm, y_train)
# resultado_decisionTree=decision_tree(X_train_norm, y_train)
#resultado_xboost=xboost(X_train_norm, y_train)

# dict_modelos={'Regressao':resultado_reg, 
#               'Random Forest':resultado_randomForest, 
#               'Ada Boost':resultado_adaBoost, 
#               'Decision Tree':resultado_decisionTree}



In [51]:
from sklearn.metrics import mean_squared_error
predict=resultado_reg.predict(X_test_norm)
# for modelo in dict_modelos.values:
#     predict=modelo.predict(X_test_norm)
#     print(predict)
#     mse= mean_squared_error(y_test, predict)
    


In [52]:
real=y_test
previsto=predict
delta=selected_columns_df['Delta_t']
real.shape, previsto.shape, delta.shape
df_reg=pd.DataFrame(data={'Predições':previsto, 'Actuals':real})
df_reg.head()



,Predições,Actuals
8064,-1.925963,0.000000
8065,1.081728,0.001666
8066,-8.483484,0.006649
8067,-15.981850,0.014584
8068,22.712145,0.024601


In [56]:
from sklearn.metrics import mean_squared_error
import numpy as np
mse = mean_squared_error(y_test, predict)

print("Mean Squared Error (MSE):", mse) #255125.98119969724
                                        # ^2 : 77724.16844499519
                                        # ^5: 50284.25539719081
                                        # xboost ^5:65159
                                        # Reg ^5: 65365
                                        # xboost x+y : 36621
                                        # regressao: x+y: 269544
                                        # regressao ^5 z: 4337
                                        # z: x+y : 4038
                                        # z: 4037
                                        
print(f'RMSE:{rmse}')
rmse = np.sqrt(mse)
max_real = df_reg['Actuals'].max
print(f'Mean Real: {mean_real}')
rmse_percentage = (rmse / mean_real) * 100
print(f"RMSE em relação à média dos valores reais: {rmse_percentage:.2f}%")
                                        

Mean Squared Error (MSE): 4337.128392423861
RMSE:65.85687809503166
Mean Real: 0.05998003797358332
RMSE em relação à média dos valores reais: 109797.99%


In [ ]:
import matplotlib.pyplot as plt

num_samples = 16128
delta_t = X_test['Delta_t'][:num_samples]
real = df_reg['Actuals'][:num_samples]
previsto = df_reg['Predições'][:num_samples]

plt.figure(figsize=(12, 6))
plt.scatter(delta_t, real, color='orange', label='Real', marker='o')
plt.scatter(delta_t, previsto, color='blue', label='Predições', marker='x')


plt.xlabel('Delta_t')
plt.ylabel('Valores')
plt.title('Valores Reais vs. Valores Previstos')
plt.legend()


plt.grid(True)
plt.show()